In [1]:
# load packages
import os
import nibabel as nib
from nilearn.image import get_data, new_img_like
from nilearn import datasets, image, signal
from nilearn.maskers import NiftiLabelsMasker,  NiftiMasker
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix
from sklearn.metrics.pairwise import pairwise_distances
import pandas as pd
import numpy as np
from scipy.stats import zscore
from tqdm import tqdm

In [2]:
# load atlas and create masks
atlas = datasets.fetch_atlas_schaefer_2018(n_rois=400, yeo_networks=17, resolution_mm=2)
atlas_img = image.load_img(atlas.maps)
masker = NiftiLabelsMasker(labels_img=atlas.maps)
masklist = []
for i in tqdm(range(1, len(atlas.labels))):
    roi_mask = image.math_img(f'img == {i}', img=atlas.maps)
    masklist.append(NiftiMasker(mask_img=roi_mask, standardize=False))
nroi = len(masklist)

[fetch_atlas_schaefer_2018] Dataset found in C:\Users\mthor\nilearn_data\schaefer_2018


100%|████████████████████████████████████████████████████████████████████████████████| 400/400 [00:21<00:00, 18.77it/s]


In [3]:
# load data
ns = 28
nc = 25
nm = 2
rdmmat = np.zeros((ns,nroi,nc,nc))

movies = ["physical", "stutterer"]
fdir = "M:/inarr/mri/"

for i in tqdm(range(ns)):
    conlist = []
    fname = fdir + "sub-" + str(i+1) + "/contrasts/" + movies[0] + ".nii.gz"
    img1 = image.load_img(fname)
    img1 = image.resample_img(img1, target_affine=atlas_img.affine, target_shape=atlas_img.shape[:3], force_resample=True, copy_header=True)
    
    conlist = []
    fname = fdir + "sub-" + str(i+1) + "/contrasts/" + movies[1] + ".nii.gz"
    img2 = image.load_img(fname)
    img2 = image.resample_img(img2, target_affine=atlas_img.affine, target_shape=atlas_img.shape[:3], force_resample=True, copy_header=True)    
    for m in range(nroi):
        roi1 = masklist[m].fit_transform(img1)
        roi2 = masklist[m].fit_transform(img2)
        rdm = pairwise_distances(roi1,roi2,metric="correlation")
        #rdm = rdm[np.tril_indices_from(rdm, k=-1)]
        #rdm = zscore(rdm)
        rdmmat[i,m,:,:] = rdm
np.save('rdm_cross.npy', rdmmat)

100%|███████████████████████████████████████████████████████████████████████████████| 28/28 [3:49:37<00:00, 492.07s/it]
